# Biomedical Image Segmentation with U-Net

In this code example, we apply the U-Net architecture to segment brain tumors from raw MRI scans as shown below. With relatively little data we are able to train a U-Net model to accurately predict where tumors exist. 

The Dice coefficient (the standard metric for the BraTS dataset used in the study) for this dataset on Unet model is about 0.82-0.88 on the large BraTs dataset.  Menze et al. [reported](http://ieeexplore.ieee.org/document/6975210/) that expert neuroradiologists manually segmented these tumors with a cross-rater Dice score of 0.75-0.85, meaning that the model’s predictions are on par with what expert physicians have made.

<img src="images/figure1.png">

Since its introduction two years ago, the [U-Net](https://arxiv.org/pdf/1505.04597.pdf0) architecture has been used to create deep learning models for segmenting [nerves](https://github.com/jocicmarko/ultrasound-nerve-segmentation) in ultrasound images, [lungs](https://www.kaggle.com/c/data-science-bowl-2017#tutorial) in CT scans, and even [interference](https://github.com/jakeret/tf_unet) in radio telescopes.

## What is U-Net?
U-Net is designed like an [auto-encoder](https://en.wikipedia.org/wiki/Autoencoder). It has an encoding path (“contracting”) paired with a decoding path (“expanding”) which gives it the “U” shape.  However, in contrast to the autoencoder, U-Net predicts a pixelwise segmentation map of the input image rather than classifying the input image as a whole. For each pixel in the original image, it asks the question: “To which class does this pixel belong?” This flexibility allows U-Net to predict different parts of the tumor simultaneously.

<img src="images/unet.png">

This module loads the data from `data.py`, creates a TensorFlow/Keras model from `model.py`, trains the model on the data, and then saves the best model.

In [ ]:
import tensorflow as tf 
import datetime
import time
import os
from IPython.display import Image

import numpy as np

import settings    # Use the custom settings.py file for default parameters

import sys; sys.argv=['']; del sys
# from model import load_model, get_callbacks, evaluate_model
from data import load_data
from model import unet

from argparser import args

if args.keras_api:
    import keras as K
else:
    from tensorflow import keras as K

onnx = False # Set whether we are exporting to ONNX model and using nGraph

For best CPU speed, Tune the number of intra and inter threads to take advantage of multi-core systems base don your hardware. Besides these settings, other important parameters are OMP_NUM_THREADS, KMP_BLOCKTIME, KMP_AFFINITY,KMP_SETTINGS that can set as environment variables. Refer the article [Maximize TensorFlow* Performance on CPU](https://software.intel.com/en-us/articles/maximize-tensorflow-performance-on-cpu-considerations-and-recommendations-for-inference) to learn more about these parameters. Set the OMP_NUM_THREADS and intra_op_parallelism_threads to number of physical cores in your processor


In [2]:

os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"
os.environ["OMP_NUM_THREADS"]= "56"# <Tune this to get best performance anywhere from 1 to # of physical cores>
args.num_threads=2
args.num_inter_threads=56 # <Tune this to get best performance anywhere from 1 to # of physical cores>

config = tf.ConfigProto(intra_op_parallelism_threads=args.num_threads,
                            inter_op_parallelism_threads=args.num_inter_threads)
sess = tf.Session(config=config)
K.backend.set_session(sess)


## Training Time!

The bulk of the training section can be broken down in 4 simple steps:
1. Load the training data
1. Define the model
3. Train the model on the data
4. Evaluate the best model
    

#### Step 1 : Loading the datafrom the `HDF5` file

In [ ]:
hdf5_filename = os.path.join(args.data_path, args.data_filename)
print ("Full path to the HFS file:", hdf5_filename)

In [ ]:
imgs_train, msks_train, imgs_validation, msks_validation, imgs_testing, msks_testing = \
    load_data(hdf5_filename, args.batch_size,[args.crop_dim, args.crop_dim])

np.random.seed(816)

#### Step 2: Define the model

In [ ]:
print("-" * 30)
print("Creating and compiling model ...")
print("-" * 30)
unet_model = unet()
model = unet_model.create_model(imgs_train.shape, msks_train.shape)

model_filename, model_callbacks = unet_model.get_callbacks()


#### Step 3: Train the model on the data

**NOTE:** The default EPOCH is 30. For testing purpose, this script trains the small BraTs dataset with 3 epochs.

To train the unet model realistically, comment the `args.epoch=3` line, or change the epoch anywhere between 20 to 30 while running on the large dataset. It is recommended to run this experiment on a high end Intel Xeon processor if you want to use the large BraTs dataset with 30 epochs, also, make sure to have a large memory space. ETA for the training job on the large BraTs dataset might be 6+ hours

In [ ]:
print("-" * 30)
print("Fitting model with training data ...")
print("-" * 30)

print("Step 3, training the model started at {}".format(datetime.datetime.now()))
start_time = time.time()

args.epochs=3

history = model.fit(imgs_train, msks_train,
          batch_size=args.batch_size,
          epochs=args.epochs,
          validation_data=(imgs_validation, msks_validation),
          verbose=1, shuffle="batch",
          callbacks=model_callbacks)

print("Total time elapsed for training = {} seconds".format(time.time() - start_time))
print("Training finished at {}".format(datetime.datetime.now()))
    
# Append training log
# with open("training.log","a+") as fp:
#     fp.write("{}: {}\n".format(datetime.datetime.now(),
#                              history.history["val_dice_coef"]))

#### Step 4: Evaluate the best model

In [ ]:
print("-" * 30)
print("Loading the best trained model ...")
print("-" * 30)
unet_model.evaluate_model(model_filename, imgs_testing, msks_testing)

## End: In this tutorial, you have learnt:
* What is the U-Net model
* How to tweak a series of environment variables to get better performance out of MKLDNN


`Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License. SPDX-License-Identifier: EPL-2.0`

`Copyright (c) 2019 Intel Corporation`